Aqui montaremos a rede para avaliar os dados para a tarefa de classificação multiclasse com a utilização dos dados brutos

1º Importaremos as bibliotecas e os dados

In [1]:
import json

from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Carregar os dados
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

2º Faremos o pré-processamento dos dados

In [2]:
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

3º Construíremos o modelo

In [3]:
# Construir o modelo
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4º Compilaremos o modelo

In [4]:
# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [5]:
# HIPERPARÂMETROS
hiperparametros = {
    "arquitetura": {
        "camadas": [
            {"tipo": "Conv2D", "filtros": 32, "kernel_size": 3, "ativacao": "relu"},
            {"tipo": "MaxPooling2D", "pool_size": 2},
            {"tipo": "Conv2D", "filtros": 64, "kernel_size": 3, "ativacao": "relu"},
            {"tipo": "MaxPooling2D", "pool_size": 2},
            {"tipo": "Flatten"},
            {"tipo": "Dense", "unidades": 64, "ativacao": "relu"},
            {"tipo": "Dense", "unidades": 10, "ativacao": "softmax"}
        ]
    },
    "inicializacao": {
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "metrics": ["accuracy"]
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

5º Treinaremos o modelo

In [6]:
# Treinar o modelo
historico = model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8305 - loss: 0.5254
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9760 - loss: 0.0782
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9819 - loss: 0.0566
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9846 - loss: 0.0488
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9875 - loss: 0.0396


- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [7]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(test_images)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


6º Testaremos o modelo

In [8]:
# Avaliar o modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9901 - loss: 0.0304
Test accuracy: 0.9919000267982483


NOTAS:

Para criar um modelo de CNN (Rede Neural Convolucional) para o conjunto de dados MNIST, que é um conjunto de dígitos manuscritos de 0 a 9, e portanto um problema de classificação multiclasse, seguiremos os seguintes passos em pseudocódigo:

Importar as bibliotecas necessárias.
Carregar o conjunto de dados MNIST.
Pré-processar os dados:
Normalizar os valores dos pixels para o intervalo de 0 a 1.
Redimensionar as imagens, se necessário, para garantir que todas tenham o mesmo tamanho.
Converter os rótulos (labels) para uma representação "one-hot" para a classificação multiclasse.
Construir o modelo da CNN:
Adicionar camadas convolucionais para extrair características das imagens.
Utilizar camadas de pooling para reduzir a dimensionalidade.
Adicionar camadas de dropout para reduzir o overfitting.
Incluir uma camada achatada (Flatten) para transformar os mapas de características em um vetor.
Adicionar uma ou mais camadas densas (fully connected) para a classificação.
A última camada densa deve ter um número de neurônios igual ao número de classes (10 para MNIST) e utilizar a função de ativação softmax para a classificação multiclasse.
Compilar o modelo:
Escolher um otimizador (por exemplo, Adam).
Definir a função de perda (loss function) como 'categorical_crossentropy' para classificação multiclasse.
Especificar métricas para monitorar, como 'accuracy'.
Treinar o modelo com os dados de treinamento.
Avaliar o modelo com os dados de teste para verificar a precisão.
Agora, vamos transformar esse pseudocódigo em código Python usando TensorFlow e Keras: